# Day 20 - claude

In [1]:
from collections import deque, defaultdict
from typing import List, Tuple, Set, Dict
import heapq

def read_maze(filename: str) -> List[List[str]]:
    with open(filename, 'r') as f:
        return [list(line.strip()) for line in f]

def find_position(maze: List[List[str]], char: str) -> Tuple[int, int]:
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == char:
                return (i, j)
    return (-1, -1)

def get_neighbors(pos: Tuple[int, int], maze: List[List[str]], allow_walls: bool = False) -> List[Tuple[int, int]]:
    rows, cols = len(maze), len(maze[0])
    r, c = pos
    neighbors = []
    
    for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        new_r, new_c = r + dr, c + dc
        if (0 <= new_r < rows and 0 <= new_c < cols and 
            (maze[new_r][new_c] != '#' or allow_walls)):
            neighbors.append((new_r, new_c))
    
    return neighbors

def shortest_path(maze: List[List[str]], start: Tuple[int, int], end: Tuple[int, int]) -> int:
    queue = deque([(start, 0)])
    visited = {start}
    
    while queue:
        pos, dist = queue.popleft()
        if pos == end:
            return dist
            
        for next_pos in get_neighbors(pos, maze):
            if next_pos not in visited:
                visited.add(next_pos)
                queue.append((next_pos, dist + 1))
    
    return float('inf')

def find_cheating_paths(maze: List[List[str]], start: Tuple[int, int], end: Tuple[int, int], 
                       normal_distance: int) -> Dict[int, int]:
    rows, cols = len(maze), len(maze[0])
    savings = defaultdict(int)
    
    # For each possible cheat start position
    for r in range(rows):
        for c in range(cols):
            if maze[r][c] == '#':
                continue
            
            start_pos = (r, c)
            start_dist = shortest_path(maze, start, start_pos)
            if start_dist == float('inf'):
                continue
                
            # Try all possible positions reachable within 2 moves (including through walls)
            visited = set()
            queue = [(0, start_pos)]
            
            while queue:
                moves, pos = queue.pop(0)
                if moves > 2:
                    break
                    
                if pos not in visited:
                    visited.add(pos)
                    
                    # If we're on a valid position (not a wall), try ending the cheat here
                    if maze[pos[0]][pos[1]] != '#':
                        end_dist = shortest_path(maze, pos, end)
                        if end_dist != float('inf'):
                            total_dist = start_dist + moves + end_dist
                            if total_dist < normal_distance:
                                saved = normal_distance - total_dist
                                savings[saved] += 1
                    
                    # Continue searching through neighbors (including walls)
                    if moves < 2:
                        for next_pos in get_neighbors(pos, maze, allow_walls=True):
                            queue.append((moves + 1, next_pos))
    
    return savings

def solve(filename: str) -> int:
    maze = read_maze(filename)
    start = find_position(maze, 'S')
    end = find_position(maze, 'E')
    
    # Find normal shortest path length
    normal_distance = shortest_path(maze, start, end)
    
    # Find all possible cheating paths and their time savings
    savings = find_cheating_paths(maze, start, end, normal_distance)
    
    # Count cheats that save at least 100 picoseconds
    return sum(count for saved, count in savings.items() if saved >= 100)

if __name__ == "__main__":
    result = solve("input.txt")
    print(f"Number of cheats saving at least 100 picoseconds: {result}")

KeyboardInterrupt: 

In [2]:
from collections import deque, defaultdict
from typing import List, Tuple, Set, Dict

def read_maze(filename: str) -> List[List[str]]:
    with open(filename, 'r') as f:
        return [list(line.strip()) for line in f]

def find_position(maze: List[List[str]], char: str) -> Tuple[int, int]:
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == char:
                return (i, j)
    return (-1, -1)

def get_distances_from_point(maze: List[List[str]], start: Tuple[int, int]) -> Dict[Tuple[int, int], int]:
    """Calculate distances to all reachable points from start using BFS"""
    distances = {start: 0}
    queue = deque([start])
    rows, cols = len(maze), len(maze[0])
    
    while queue:
        r, c = queue.popleft()
        curr_dist = distances[(r, c)]
        
        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_r, new_c = r + dr, c + dc
            if (0 <= new_r < rows and 0 <= new_c < cols and 
                maze[new_r][new_c] != '#' and 
                (new_r, new_c) not in distances):
                distances[(new_r, new_c)] = curr_dist + 1
                queue.append((new_r, new_c))
    
    return distances

def solve(filename: str) -> int:
    maze = read_maze(filename)
    rows, cols = len(maze), len(maze[0])
    start = find_position(maze, 'S')
    end = find_position(maze, 'E')
    
    # Pre-calculate distances from start and end to all points
    start_distances = get_distances_from_point(maze, start)
    end_distances = get_distances_from_point(maze, end)
    
    # Normal shortest path length
    normal_distance = start_distances.get(end, float('inf'))
    if normal_distance == float('inf'):
        return 0  # No valid path exists
        
    savings = defaultdict(int)
    
    # For each valid position as cheat start
    for r1 in range(rows):
        for c1 in range(cols):
            if maze[r1][c1] == '#':
                continue
                
            start_pos = (r1, c1)
            if start_pos not in start_distances:
                continue
                
            start_dist = start_distances[start_pos]
            
            # Try all positions within 2 moves (including through walls)
            for r2 in range(max(0, r1-2), min(rows, r1+3)):
                for c2 in range(max(0, c1-2), min(cols, c1+3)):
                    if maze[r2][c2] == '#':
                        continue
                        
                    end_pos = (r2, c2)
                    # Check if position is reachable within exactly 2 moves
                    moves = abs(r2-r1) + abs(c2-c1)
                    if moves > 2:
                        continue
                        
                    if end_pos not in end_distances:
                        continue
                        
                    end_dist = end_distances[end_pos]
                    total_dist = start_dist + moves + end_dist
                    
                    if total_dist < normal_distance:
                        saved = normal_distance - total_dist
                        savings[saved] += 1
    
    # Count cheats that save at least 100 picoseconds
    return sum(count for saved, count in savings.items() if saved >= 100)

if __name__ == "__main__":
    result = solve("input.txt")
    print(f"Number of cheats saving at least 100 picoseconds: {result}")

Number of cheats saving at least 100 picoseconds: 1402


## Part 2

In [3]:
from collections import deque, defaultdict
from typing import List, Tuple, Set, Dict
import heapq

def read_maze(filename: str) -> List[List[str]]:
    with open(filename, 'r') as f:
        return [list(line.strip()) for line in f]

def find_position(maze: List[List[str]], char: str) -> Tuple[int, int]:
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == char:
                return (i, j)
    return (-1, -1)

def get_distances_from_point(maze: List[List[str]], start: Tuple[int, int]) -> Dict[Tuple[int, int], int]:
    """Calculate distances to all reachable points from start using BFS"""
    distances = {start: 0}
    queue = deque([start])
    rows, cols = len(maze), len(maze[0])
    
    while queue:
        r, c = queue.popleft()
        curr_dist = distances[(r, c)]
        
        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_r, new_c = r + dr, c + dc
            if (0 <= new_r < rows and 0 <= new_c < cols and 
                maze[new_r][new_c] != '#' and 
                (new_r, new_c) not in distances):
                distances[(new_r, new_c)] = curr_dist + 1
                queue.append((new_r, new_c))
    
    return distances

def find_all_reachable_positions(maze: List[List[str]], start: Tuple[int, int], max_moves: int) -> Dict[Tuple[int, int], int]:
    """Find all positions reachable within max_moves steps, including through walls"""
    rows, cols = len(maze), len(maze[0])
    distances = {start: 0}
    queue = [(0, start)]
    
    while queue:
        dist, pos = heapq.heappop(queue)
        if dist > max_moves:
            break
            
        r, c = pos
        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_r, new_c = r + dr, c + dc
            new_pos = (new_r, new_c)
            
            if (0 <= new_r < rows and 0 <= new_c < cols and 
                new_pos not in distances):
                distances[new_pos] = dist + 1
                heapq.heappush(queue, (dist + 1, new_pos))
    
    return distances

def solve(filename: str) -> int:
    maze = read_maze(filename)
    rows, cols = len(maze), len(maze[0])
    start = find_position(maze, 'S')
    end = find_position(maze, 'E')
    
    # Pre-calculate distances from start and end to all points (without cheating)
    start_distances = get_distances_from_point(maze, start)
    end_distances = get_distances_from_point(maze, end)
    
    # Normal shortest path length
    normal_distance = start_distances.get(end, float('inf'))
    if normal_distance == float('inf'):
        return 0  # No valid path exists
    
    # Track unique cheats by recording best savings for each (start, end) pair
    best_savings = {}
    
    # For each valid position as cheat start
    for r1 in range(rows):
        for c1 in range(cols):
            if maze[r1][c1] == '#':
                continue
                
            start_pos = (r1, c1)
            if start_pos not in start_distances:
                continue
                
            start_dist = start_distances[start_pos]
            
            # Find all positions reachable within 20 moves through walls
            cheat_distances = find_all_reachable_positions(maze, start_pos, 20)
            
            # For each potential end position
            for end_pos, cheat_moves in cheat_distances.items():
                if maze[end_pos[0]][end_pos[1]] == '#':
                    continue
                    
                if end_pos not in end_distances:
                    continue
                    
                end_dist = end_distances[end_pos]
                total_dist = start_dist + cheat_moves + end_dist
                
                if total_dist < normal_distance:
                    saved = normal_distance - total_dist
                    cheat_key = (start_pos, end_pos)
                    
                    # Only keep the best saving for each unique start-end pair
                    if cheat_key not in best_savings or saved > best_savings[cheat_key]:
                        best_savings[cheat_key] = saved
    
    # Count cheats that save at least 100 picoseconds
    savings_count = defaultdict(int)
    for saved in best_savings.values():
        savings_count[saved] += 1
        
    return sum(count for saved, count in savings_count.items() if saved >= 100)

if __name__ == "__main__":
    result = solve("input.txt")
    print(f"Number of cheats saving at least 100 picoseconds: {result}")

Number of cheats saving at least 100 picoseconds: 1153545


In [4]:
from collections import deque, defaultdict
from typing import List, Tuple, Set, Dict
import heapq

def read_maze(filename: str) -> List[List[str]]:
    with open(filename, 'r') as f:
        return [list(line.strip()) for line in f]

def find_position(maze: List[List[str]], char: str) -> Tuple[int, int]:
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == char:
                return (i, j)
    return (-1, -1)

def manhattan_distance(a: Tuple[int, int], b: Tuple[int, int]) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def get_distances_from_point(maze: List[List[str]], start: Tuple[int, int]) -> Dict[Tuple[int, int], int]:
    """Calculate distances to all reachable points from start using BFS"""
    distances = {start: 0}
    queue = deque([start])
    rows, cols = len(maze), len(maze[0])
    
    while queue:
        r, c = queue.popleft()
        curr_dist = distances[(r, c)]
        
        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_r, new_c = r + dr, c + dc
            if (0 <= new_r < rows and 0 <= new_c < cols and 
                maze[new_r][new_c] != '#' and 
                (new_r, new_c) not in distances):
                distances[(new_r, new_c)] = curr_dist + 1
                queue.append((new_r, new_c))
    
    return distances

def find_best_cheat_path(maze: List[List[str], 
                        start_pos: Tuple[int, int], 
                        target: Tuple[int, int], 
                        max_moves: int) -> Tuple[int, Tuple[int, int]]:
    """Find shortest path from start to any valid position using A* with limited moves"""
    rows, cols = len(maze), len(maze[0])
    
    # Priority queue entries: (f_score, moves_used, position)
    queue = [(manhattan_distance(start_pos, target), 0, start_pos)]
    # Track shortest path to each position
    best_moves = {start_pos: 0}
    # Track best end position found
    best_end = None
    best_moves_used = float('inf')
    
    while queue:
        _, moves_used, pos = heapq.heappop(queue)
        
        if moves_used > max_moves:
            continue
            
        # If we're on a valid position (not a wall), consider it as potential end
        if maze[pos[0]][pos[1]] != '#':
            if best_end is None or moves_used < best_moves_used:
                best_end = pos
                best_moves_used = moves_used
        
        if moves_used == max_moves:
            continue
            
        # Try all directions
        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_r, new_c = pos[0] + dr, pos[1] + dc
            new_pos = (new_r, new_c)
            
            if (0 <= new_r < rows and 0 <= new_c < cols and 
                (new_pos not in best_moves or moves_used + 1 < best_moves[new_pos])):
                best_moves[new_pos] = moves_used + 1
                f_score = moves_used + 1 + manhattan_distance(new_pos, target)
                heapq.heappush(queue, (f_score, moves_used + 1, new_pos))
    
    return (best_moves_used, best_end) if best_end is not None else (float('inf'), None)

def solve(filename: str) -> int:
    maze = read_maze(filename)
    rows, cols = len(maze), len(maze[0])
    start = find_position(maze, 'S')
    end = find_position(maze, 'E')
    
    # Pre-calculate distances from start and end
    start_distances = get_distances_from_point(maze, start)
    end_distances = get_distances_from_point(maze, end)
    
    # Normal shortest path length
    normal_distance = start_distances.get(end, float('inf'))
    if normal_distance == float('inf'):
        return 0
    
    # Track unique cheats by their start and end positions
    best_savings = {}
    
    # For each valid position as cheat start
    for r in range(rows):
        for c in range(cols):
            start_pos = (r, c)
            if maze[r][c] == '#' or start_pos not in start_distances:
                continue
                
            # Use A* to find best path towards end
            cheat_moves, cheat_end = find_best_cheat_path(maze, start_pos, end, 20)
            
            if cheat_end is None or cheat_end not in end_distances:
                continue
            
            # Calculate total path length with this cheat
            start_dist = start_distances[start_pos]  # Distance from S to cheat start
            end_dist = end_distances[cheat_end]     # Distance from cheat end to E
            total_dist = start_dist + cheat_moves + end_dist
            
            if total_dist < normal_distance:
                saved = normal_distance - total_dist
                cheat_key = (start_pos, cheat_end)
                
                # Only keep the best saving for each unique start-end pair
                if cheat_key not in best_savings or saved > best_savings[cheat_key]:
                    best_savings[cheat_key] = saved
    
    # Count cheats that save at least 100 picoseconds
    return sum(1 for saved in best_savings.values() if saved >= 100)

if __name__ == "__main__":
    result = solve("input.txt")
    print(f"Number of cheats saving at least 100 picoseconds: {result}")

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' on line 39 (768782361.py, line 42)

In [5]:
from collections import deque, defaultdict
from typing import List, Tuple, Set, Dict
import heapq

def read_maze(filename: str) -> List[List[str]]:
    with open(filename, 'r') as f:
        return [list(line.strip()) for line in f]

def find_position(maze: List[List[str]], char: str) -> Tuple[int, int]:
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == char:
                return (i, j)
    return (-1, -1)

def manhattan_distance(a: Tuple[int, int], b: Tuple[int, int]) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def get_distances_from_point(maze: List[List[str]], start: Tuple[int, int]) -> Dict[Tuple[int, int], int]:
    """Calculate distances to all reachable points from start using BFS"""
    distances = {start: 0}
    queue = deque([start])
    rows, cols = len(maze), len(maze[0])
    
    while queue:
        r, c = queue.popleft()
        curr_dist = distances[(r, c)]
        
        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_r, new_c = r + dr, c + dc
            if (0 <= new_r < rows and 0 <= new_c < cols and 
                maze[new_r][new_c] != '#' and 
                (new_r, new_c) not in distances):
                distances[(new_r, new_c)] = curr_dist + 1
                queue.append((new_r, new_c))
    
    return distances

def find_best_cheat_path(maze: List[List[str]], 
                        start_pos: Tuple[int, int], 
                        target: Tuple[int, int], 
                        max_moves: int) -> Tuple[int, Tuple[int, int]]:
    """Find shortest path from start to any valid position using A* with limited moves"""
    rows, cols = len(maze), len(maze[0])
    
    # Priority queue entries: (f_score, moves_used, position)
    queue = [(manhattan_distance(start_pos, target), 0, start_pos)]
    # Track shortest path to each position
    best_moves = {start_pos: 0}
    # Track best end position found
    best_end = None
    best_moves_used = float('inf')
    
    while queue:
        _, moves_used, pos = heapq.heappop(queue)
        
        if moves_used > max_moves:
            continue
            
        # If we're on a valid position (not a wall), consider it as potential end
        if maze[pos[0]][pos[1]] != '#':
            if best_end is None or moves_used < best_moves_used:
                best_end = pos
                best_moves_used = moves_used
        
        if moves_used == max_moves:
            continue
            
        # Try all directions
        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_r, new_c = pos[0] + dr, pos[1] + dc
            new_pos = (new_r, new_c)
            
            if (0 <= new_r < rows and 0 <= new_c < cols and 
                (new_pos not in best_moves or moves_used + 1 < best_moves[new_pos])):
                best_moves[new_pos] = moves_used + 1
                f_score = moves_used + 1 + manhattan_distance(new_pos, target)
                heapq.heappush(queue, (f_score, moves_used + 1, new_pos))
    
    return (best_moves_used, best_end) if best_end is not None else (float('inf'), None)

def solve(filename: str) -> int:
    maze = read_maze(filename)
    rows, cols = len(maze), len(maze[0])
    start = find_position(maze, 'S')
    end = find_position(maze, 'E')
    
    # Pre-calculate distances from start and end
    start_distances = get_distances_from_point(maze, start)
    end_distances = get_distances_from_point(maze, end)
    
    # Normal shortest path length
    normal_distance = start_distances.get(end, float('inf'))
    if normal_distance == float('inf'):
        return 0
    
    # Track unique cheats by their start and end positions
    best_savings = {}
    
    # For each valid position as cheat start
    for r in range(rows):
        for c in range(cols):
            start_pos = (r, c)
            if maze[r][c] == '#' or start_pos not in start_distances:
                continue
                
            # Use A* to find best path towards end
            cheat_moves, cheat_end = find_best_cheat_path(maze, start_pos, end, 20)
            
            if cheat_end is None or cheat_end not in end_distances:
                continue
            
            # Calculate total path length with this cheat
            start_dist = start_distances[start_pos]  # Distance from S to cheat start
            end_dist = end_distances[cheat_end]     # Distance from cheat end to E
            total_dist = start_dist + cheat_moves + end_dist
            
            if total_dist < normal_distance:
                saved = normal_distance - total_dist
                cheat_key = (start_pos, cheat_end)
                
                # Only keep the best saving for each unique start-end pair
                if cheat_key not in best_savings or saved > best_savings[cheat_key]:
                    best_savings[cheat_key] = saved
    
    # Count cheats that save at least 100 picoseconds
    return sum(1 for saved in best_savings.values() if saved >= 100)

if __name__ == "__main__":
    result = solve("input.txt")
    print(f"Number of cheats saving at least 100 picoseconds: {result}")

Number of cheats saving at least 100 picoseconds: 0


In [6]:
from collections import deque, defaultdict
from typing import List, Tuple, Set, Dict
import heapq

def read_maze(filename: str) -> List[List[str]]:
    with open(filename, 'r') as f:
        return [list(line.strip()) for line in f]

def find_position(maze: List[List[str]], char: str) -> Tuple[int, int]:
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == char:
                return (i, j)
    return (-1, -1)

def get_distances_from_point(maze: List[List[str]], start: Tuple[int, int]) -> Dict[Tuple[int, int], int]:
    """Calculate distances to all reachable points from start using BFS"""
    distances = {start: 0}
    queue = deque([start])
    rows, cols = len(maze), len(maze[0])
    
    while queue:
        r, c = queue.popleft()
        curr_dist = distances[(r, c)]
        
        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_r, new_c = r + dr, c + dc
            if (0 <= new_r < rows and 0 <= new_c < cols and 
                maze[new_r][new_c] != '#' and 
                (new_r, new_c) not in distances):
                distances[(new_r, new_c)] = curr_dist + 1
                queue.append((new_r, new_c))
    
    return distances

def find_cheat_destinations(maze: List[List[str]], start_pos: Tuple[int, int], max_moves: int) -> Dict[Tuple[int, int], int]:
    """Find all reachable non-wall positions within max_moves steps"""
    rows, cols = len(maze), len(maze[0])
    distances = {}
    visited = set()
    queue = deque([(start_pos, 0)])
    
    while queue:
        pos, moves = queue.popleft()
        r, c = pos
        
        if moves > max_moves:
            continue
            
        if maze[r][c] != '#':
            if pos not in distances or moves < distances[pos]:
                distances[pos] = moves
        
        if moves == max_moves:
            continue
            
        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_r, new_c = r + dr, c + dc
            new_pos = (new_r, new_c)
            
            if (0 <= new_r < rows and 0 <= new_c < cols and 
                (new_pos, moves + 1) not in visited):
                visited.add((new_pos, moves + 1))
                queue.append((new_pos, moves + 1))
    
    return distances

def solve(filename: str) -> int:
    maze = read_maze(filename)
    rows, cols = len(maze), len(maze[0])
    start = find_position(maze, 'S')
    end = find_position(maze, 'E')
    
    # Pre-calculate distances from start and end
    start_distances = get_distances_from_point(maze, start)
    end_distances = get_distances_from_point(maze, end)
    
    # Normal shortest path length
    normal_distance = start_distances[end]
    
    # Track unique cheats by their start and end positions
    best_savings = {}
    
    # For each valid position as cheat start
    for r in range(rows):
        for c in range(cols):
            start_pos = (r, c)
            if maze[r][c] == '#' or start_pos not in start_distances:
                continue
            
            # Distance to reach cheat start point
            start_dist = start_distances[start_pos]
            
            # Find all possible cheat destinations
            cheat_destinations = find_cheat_destinations(maze, start_pos, 20)
            
            # Check each possible cheat end point
            for end_pos, cheat_moves in cheat_destinations.items():
                if end_pos not in end_distances:
                    continue
                
                # Calculate full path length with this cheat
                end_dist = end_distances[end_pos]
                total_dist = start_dist + cheat_moves + end_dist
                
                if total_dist < normal_distance:
                    saved = normal_distance - total_dist
                    cheat_key = (start_pos, end_pos)
                    
                    # Only keep the best saving for each unique start-end pair
                    if cheat_key not in best_savings or saved > best_savings[cheat_key]:
                        best_savings[cheat_key] = saved
    
    # Count cheats that save at least 100 picoseconds
    return sum(1 for saved in best_savings.values() if saved >= 100)

if __name__ == "__main__":
    result = solve("input.txt")
    print(f"Number of cheats saving at least 100 picoseconds: {result}")

Number of cheats saving at least 100 picoseconds: 1020244
